<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W5/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import os
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
from keras.layers import InputLayer
from keras.layers import Embedding
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn import model_selection
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

device=torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

data=pd.read_csv("xaa",encoding="utf-8")

vect  = CountVectorizer()
#corpus = vect.fit_transform(data["text"])
#train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data["label"],test_size=0.4)
#Encoder = LabelEncoder()
#train_label = Encoder.fit_transform(train_label)
#test_label = Encoder.fit_transform(test_label)
#train_corpus_vect=vect.transform(train_corpus)
#test_corpus_vect=vect.transform(test_corpus)

# Initalise vect.vocabulary_
vect.fit_transform(data["text"])

maxlen=0

def transform(text,vect):
    global maxlen
    d=vect.vocabulary_
    p=vect.build_preprocessor()
    t=vect.build_tokenizer()
    vec_list=[]
    for doc in text:
        tokens=t(p(doc))
        s=len(tokens)
        if s>maxlen:
            maxlen=s
        doc_vec=np.array([d[token] for token in tokens])
        #doc_vec=sequence.pad_sequences(doc_vec,maxlen=maxlen)
        vec_list.append(doc_vec)
    vec_list=sequence.pad_sequences(vec_list,maxlen=maxlen,padding="post")
    corpus_vec=np.vstack(vec_list)
    #return nn.functional.normalize(torch.tensor(corpus_vec).float())
    return torch.tensor(corpus_vec)
    #return torch.tensor(corpus_vec).float()

# print(corpus_vec)

bsize=64
epochs=5
lr=1e-3

class corpus(Dataset):
    def __init__(self,corpus,label,seq):
        self.corpus=corpus
        self.label=label
        self.seq=seq
    def __len__(self):
        return len(self.corpus)
    def __getitem__(self,idx):
        return self.corpus[idx],self.label[idx]

class lstm(nn.Module):
    def __init__(self,input_size,hidden_size,seq):
        super(lstm,self).__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size
        self.seq=seq
        self.rnn=True
        self.lstm=nn.LSTM(input_size,hidden_size,batch_first=True,num_layers=1)
        self.fc=nn.Linear(self.hidden_size,1)
        self.embed=nn.Embedding(len(vect.vocabulary_),input_size)

    def forward(self,x,h0=None,c0=None):
        x=self.embed(x)
        if h0==None and c0==None:
            x, (hn,cn) = self.lstm(x)
        else:
            x, (hn,cn) = self.lstm(x,(h0,c0))
        #print(x[:,-1,:].shape)
        #assert False,"blah"
        x = torch.flatten(x[:,-1,:],1)
        x = self.fc(x)
        return nn.Sigmoid()(x), hn,cn
        #return nn.Softmax()(x), hn,cn

class dense(nn.Module):
    def __init__(self,seq,vocab,embed_dim):
        super(dense,self).__init__()
        self.seq=seq
        self.rnn=False
        self.network=nn.Sequential(
#            nn.Embedding(vocab,embed_dim),
            nn.Linear(embed_dim*seq,360),
            nn.ReLU(),
            nn.Linear(360,180),
            nn.ReLU(),
            nn.Linear(180,1),
            nn.Sigmoid()
        )
        self.embed=nn.Embedding(vocab,embed_dim)

    def forward(self,x):
        #print(x.shape)
        x=self.embed(x)
        x=torch.flatten(x,1)
        #x=torch.transpose(x,1,2)
        #x=torch.flatten(x,1)
        return self.network(x)

vocab=len(vect.vocabulary_)
Encoder = LabelEncoder()
corpus_vec = transform(data["text"],vect)
train_corpus_vec, test_corpus_vec, train_label, test_label = model_selection.train_test_split(corpus_vec,data["label"],test_size=0.2)
train_label = torch.from_numpy(Encoder.fit_transform(train_label)).float()
test_label = torch.from_numpy(Encoder.fit_transform(test_label)).float()
#train_corpus_vec = transform(train_corpus,vect)
#test_corpus_vec = transform(test_corpus,vect)
lstm_classifier=lstm(16,20,maxlen)
loss_fn=nn.BCELoss()
#loss_fn=nn.BCEWithLogitsLoss()
#loss_fn=nn.MSELoss()
optimizer=torch.optim.Adam(lstm_classifier.parameters(),lr=lr)
#optimizer=torch.optim.SGD(lstm_classifier.parameters(),lr=lr)
#print(len(lstm_classifier(torch.reshape(train_corpus_vec[0],(bsize,maxlen,1)))))

dense_classifier=dense(maxlen,vocab,16)

c=lstm_classifier

train_dataloader=DataLoader(corpus(train_corpus_vec,train_label,maxlen),batch_size=bsize,shuffle=True)
test_dataloader=DataLoader(corpus(test_corpus_vec,test_label,maxlen),batch_size=bsize,shuffle=True)

def train(dataloader,model,loss_fn,optimizer):
    hn,cn=None,None
    size=len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y=x.to(device),y.to(device)
        #print(torch.sum(x[0]!=0))
        if model.rnn==True:
            #pred,hn,cn=model(x.reshape(bsize,dataloader.dataset.seq,1).to(device),hn,cn)
            pred,hn,cn=model(x,hn,cn)
        else:
            pred=model(x)
        #pred=model(x)
        cost=loss_fn(pred.flatten(),y)
        cost.backward()
        #if model.rnn==True:
        #    print(model.lstm.weight_ih_l0.grad[0][0])
        optimizer.step()
        optimizer.zero_grad()

        if model.rnn==True:
            hn=hn.detach()
            cn=cn.detach()
        if batch % 10 == 0:
            cost_val, current = cost.item(), batch * bsize + len(x)
            print(f"cost: {cost_val:>7f}, accuracy: {(torch.round(pred.flatten())==y).sum().item()/bsize:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x,y=x.to(device),y.to(device)
            if model.rnn==True:
                #pred,_,_=model(x.reshape(bsize,dataloader.dataset.seq,1).to(device))
                pred,_,_=model(x)
            else:
                pred=model(x)
            #pred=model(x)
            #print(torch.round(pred.flatten()),y)
            test_loss += loss_fn(pred.flatten(), y).item()
            ncorrect = (torch.round(pred.flatten()) == y).sum().item()
            print(ncorrect)
            correct += ncorrect

    print(correct,size)
    print(f"Test Error: \n Accuracy: {(100*correct/size):>0.1f}%, Avg loss: {100*test_loss/size:>8f} \n")

#keras_model=Sequential([InputLayer(input_shape=(maxlen,),batch_size=bsize),
#                        Embedding(len(vect.vocabulary_),16),
#                  Flatten(),
#                  Dense(1,activation="sigmoid")])
keras_model=Sequential([InputLayer(input_shape=(maxlen,),batch_size=bsize),
                        Embedding(len(vect.vocabulary_),16),
                        LSTM(20,return_sequences=True),
                  Flatten(),
                  Dense(1,activation="sigmoid")])
keras_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
keras_model.summary()

use_torch=True

#print(corpus_vec[0])
if use_torch==True:
    for epoch in range(epochs):
        train(train_dataloader,c,loss_fn,optimizer)
    test_loop(test_dataloader,c,loss_fn)
else:
    keras_model.fit(train_corpus_vec,train_label,batch_size=bsize,epochs=epochs,validation_data=(test_corpus_vec,test_label))
    keras_model.evaluate(test_corpus_vec,test_label,batch_size=bsize)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "sequential_12"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)        │ (64, 191, 16)          │       769,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (64, 191, 20)          │         2,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_11 (Flatten)            │ (64, 3820)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (64, 1)                │         3,821 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 776,237 (2.96 MB)

 Trainable params: 776,237 (2.96 MB)

 Non-trainable params: 0 (0.00 B)

cost: 0.675576, accuracy: 0.593750  [   64/16000]
cost: 0.696082, accuracy: 0.500000  [  704/16000]
cost: 0.692830, accuracy: 0.515625  [ 1344/16000]
cost: 0.703220, accuracy: 0.453125  [ 1984/16000]
cost: 0.692962, accuracy: 0.546875  [ 2624/16000]
cost: 0.698966, accuracy: 0.421875  [ 3264/16000]
cost: 0.688470, accuracy: 0.562500  [ 3904/16000]
cost: 0.699651, accuracy: 0.406250  [ 4544/16000]
cost: 0.699313, accuracy: 0.421875  [ 5184/16000]
cost: 0.689933, accuracy: 0.546875  [ 5824/16000]
cost: 0.687586, accuracy: 0.578125  [ 6464/16000]
cost: 0.694030, accuracy: 0.468750  [ 7104/16000]
cost: 0.690894, accuracy: 0.562500  [ 7744/16000]
cost: 0.689893, accuracy: 0.593750  [ 8384/16000]
cost: 0.696194, accuracy: 0.421875  [ 9024/16000]
cost: 0.693260, accuracy: 0.500000  [ 9664/16000]
cost: 0.694577, accuracy: 0.484375  [10304/16000]
cost: 0.711983, accuracy: 0.343750  [10944/16000]
cost: 0.691484, accuracy: 0.531250  [11584/16000]
cost: 0.695377, accuracy: 0.453125  [12224/16000]
